In [ ]:
import pandas as pd
import numpy as np
import copy

ratings = pd.read_csv('u.data', sep='\t', header=None)
ratings

In [ ]:
R = ratings.pivot(index=0, columns=1, values=2).fillna(0).values
R

In [ ]:
A = np.array([[4,5,4,0], [3,2,0,4], [2,0,5,5]])
A

In [ ]:
maska = A>0
maska

In [ ]:
no_user = A.shape[0]
no_movies = A.shape[1]
no_l = 2 

eta = 0.01

In [ ]:
Q = np.random.random((no_user, no_l))
P = np.random.random((no_l, no_movies))
P

In [ ]:
for i in range(Q.shape[0]):
    for j in range(Q.shape[1]):
        grad = 0
        for idl, l in enumerate(A[i,:]):
            if( l ):
                grad += A[i,idl]-(Q[i,0]*P[0,idl]+Q[i,1]*P[1,idl])*P[j, idl]
        Q[i,j] = Q[i,j] + eta*grad
Q.shape[1]

In [ ]:
for j in range(P.shape[1]):
    for i in range(P.shape[0]):
        grad = 0
        for idl, l in enumerate(A[:,j]):
            if( l ):
                grad += A[idl, j]-(Q[idl,0]*P[0,j]+Q[idl,1]*P[1,j])*Q[idl, i]
        P[i,j] = P[i,j] + eta*grad
P.shape

In [ ]:
pred = np.matmul(Q, P)
MAE = np.abs(A - pred).mean()

In [ ]:
def train(Q, P, A, eta=0.01, epok=10):
    for e in range(epok):
        Q1 = np.zeros(Q.shape)
        for i in range(Q.shape[0]):
            for j in range(Q.shape[1]):
                grad = 0                
                for idl, l in enumerate(A[i,:]):
                    if( l ):
                        
                        grad += (A[i,idl]-np.dot(Q[i,:], P[:,idl]))*P[j, idl]
                grad /= sum(A[i,:]>0)
                Q1[i,j] = Q[i,j] + eta*grad

        P1 = np.zeros(P.shape)
        for j in range(P.shape[1]):
            for i in range(P.shape[0]):
                grad = 0
                for idl, l in enumerate(A[:,j]):
                    if( l ):
                        grad += (A[idl, j]-np.dot(Q[idl,:], P[:,j]))*Q[idl, i]
                grad /= sum(A[:,j]>0)
                P1[i,j] = P[i,j] + eta*grad

        Q = copy.deepcopy(Q1)
        P = copy.deepcopy(P1)
        pred = np.matmul(Q, P)
        MAE = np.abs(A - pred).mean()
        print(f'Epoka: {e} \t MAE: {MAE}')
    return Q, P

In [ ]:
pred = np.matmul(Q, P)
pred

In [ ]:
no_user = R.shape[0]
no_movies = R.shape[1]
no_l = 100 
Q = np.random.random((no_user, no_l))
P = np.random.random((no_l, no_movies))

Q, P = train(Q, P, R, 0.01, 4)


In [ ]:
Q, P = train(Q, P, R, 0.01, 100)
pred = np.matmul(Q, P)
pred

In [ ]:
maska = R>0
pred = np.matmul(Q, P)
MAE = np.abs(R - np.multiply(pred, maska)).sum() / maska.sum()
MAE